# Phase 1: Exploratory data analysis and data pre-processing

### Taryn Chung (301239706), ZeYu Zhu (301341176), Kearro Chow (301279961)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
color = sns.color_palette()

In [5]:
train_df = pd.read_json("train.json")
train_df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level
4,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/7170325_3bb5ac84...,2400,145 Borinquen Place,medium
6,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,955db33477af4f40004820b4aed804a0,[https://photos.renthop.com/2/7092344_7663c19a...,3800,230 East 44th,low
9,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,c8b10a317b766204f08e613cef4ce7a0,[https://photos.renthop.com/2/7158677_c897a134...,3495,405 East 56th Street,medium
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,medium
15,1.0,0,bfb9405149bfff42a92980b594c28234,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,-73.9743,2c3b41f588fbb5234d8a1e885a436cfa,[https://photos.renthop.com/2/7225292_901f1984...,2795,340 East 34th Street,low


### Display Address

In [6]:
whichrow = 0
count = 0
for row in train_df['display_address']:
    if any(c.isalpha() for c in row) == False:
        train_df.loc[train_df.index[whichrow], 'display_address'] = np.nan
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)

number of missing values:  141


Since there are relatively few missing values and an address is important to have, we removed these values by changing them to NAN and dropping every row with a NAN.

### Street Address

In [7]:
whichrow = 0
count = 0
for row in train_df['street_address']:
    if any(c.isalpha() for c in row) == False :
        train_df.loc[train_df.index[whichrow], 'street_address'] = np.nan
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)

number of missing values:  13


Since there are relatively few missing values and an address is important to have, we removed these values by changing them to NAN and dropping every row with a NAN.

### Latitude

In [8]:
whichrow = 0
count = 0
for row in train_df['latitude']:
    if row==0:
        train_df.loc[train_df.index[whichrow], 'latitude'] = np.nan
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)

number of missing values:  12


Since there are relatively few missing values and an coordinates are important to have, we removed these values by changing them to NAN and dropping every row with a NAN.

### Longitude

In [9]:
whichrow = 0
count = 0
for row in train_df['longitude']:
    if row==0:
        train_df.loc[train_df.index[whichrow], 'longitude'] = np.nan
        count+=1
        whichrow+=1
    else:
        whichrow += 1
print ("number of missing values: ", count)

number of missing values:  12


## Handling outliers

### Prices

In [10]:
q25, q75 = np.percentile(train_df['price'].values, 25), np.percentile(train_df['price'].values, 75)
iqr = q75-q25
cutoff = iqr * 3
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['price'].values if x > upper or x < lower]
print ("number of extreme outliers:", len(outliers))
#print(lower,upper)

number of extreme outliers: 1223


In [11]:
q25, q75 = np.percentile(train_df['price'].values, 25), np.percentile(train_df['price'].values, 75)
iqr = q75-q25
cutoff = iqr * 1.5
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['price'].values if x > upper or x < lower]
print ("number of mild outliers:", len(outliers))
#print(lower,upper)

number of mild outliers: 2788


#### Before Handling

In [0]:
# sns.boxplot(y=train_df['price'])

#### After Handling

In [13]:
from scipy import nanmean
import math

whichrow = 0
for row in train_df['price']:
    if row>upper or row<lower:
        train_df.loc[train_df.index[whichrow], 'price'] = np.nan
        whichrow+=1
    else:
        whichrow += 1
        
mean_val =round(nanmean(train_df['price']),2)

whichrow = 0
for row in train_df['price']:
    if math.isnan(row):
        train_df.loc[train_df.index[whichrow],'price'] = mean_val
        whichrow+=1
    else:
        whichrow+=1

# sns.boxplot(y=train_df['price'], whis = [0,100])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: scipy.nanmean is deprecated and will be removed in SciPy 2.0.0, use numpy.nanmean instead
  if sys.path[0] == '':


We found the Tukey inner and outer fences to find the mild and extreme outliers. Extreme fences included a cutoff that left listings with a negative price range. We decided to only remove the mild outliers instead.  We replaced them with the mean value 3272.00 of so we have more data points to train with. The upper fence cutoff was 6,500 and the lower cutoff was 100. This dealt with outliers such as listings prices at 4,490,000, 43, and negative numbers, which are unrealistic. The new minimum price is 401 and the maximum is now 6,500.

### Latitude

#### Before Handling

In [14]:
q25, q75 = np.percentile(train_df['latitude'].values, 25), np.percentile(train_df['latitude'].values, 75)
iqr = q75-q25
cutoff = iqr * 1.5
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['latitude'].values if x > upper or x < lower]
print ("number of mild outliers:", len(outliers))
#print(lower, upper)

number of mild outliers: 0


In [15]:
q25, q75 = np.percentile(train_df['latitude'].values, 25), np.percentile(train_df['latitude'].values, 75)
iqr = q75-q25
cutoff = iqr * 3
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['latitude'].values if x > upper or x < lower]
print ("number of extreme outliers:", len(outliers))
#print(lower, upper)

number of extreme outliers: 0


In [0]:
# sns.boxplot(y=train_df['latitude'])

#### After Handling

In [17]:
whichrow = 0
for row in train_df['latitude']:
    if row>upper or row<lower:
        train_df.loc[train_df.index[whichrow], 'latitude'] = np.nan
        whichrow+=1
    else:
        whichrow += 1
        
mean_val = nanmean(train_df['latitude'])
#print(mean_val)
whichrow = 0
for row in train_df['latitude']:
    if math.isnan(row):
        train_df.loc[train_df.index[whichrow],'latitude'] = mean_val
        whichrow+=1
    else:
        whichrow+=1

# sns.boxplot(y=train_df['latitude'], whis = [0,100])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: scipy.nanmean is deprecated and will be removed in SciPy 2.0.0, use numpy.nanmean instead
  if __name__ == '__main__':


We found the Tukey inner and outer fences to find the mild and extreme outliers. The inner fences still included listings that were within the NYC bounds, accoding to Google Maps. So we decided to only deal with the extreme outliers that were a little further out than NYC vicinity. We replaced with the mean value  of 40.7513098874893 so we have more data points to train with. The fence cutoffs we used were 40.5903 and 40.912299999999995. This dealt with outliers such as listings located in LA because we wanted to keep the listings within NYC. The new maximum and minimum longitude and latitude are now 40.9121 and 40.5904, respectively.

### Longitude

In [18]:
q25, q75 = np.percentile(train_df['longitude'].values, 25), np.percentile(train_df['longitude'].values, 75)
iqr = q75-q25
cutoff = iqr * 3
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['longitude'].values if x > upper or x < lower]
print ("number of extreme outliers:", len(outliers))
#print(lower, upper)

number of extreme outliers: 0


In [19]:
q25, q75 = np.percentile(train_df['longitude'].values, 25), np.percentile(train_df['longitude'].values, 75)
iqr = q75-q25
cutoff = iqr * 1.5
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['longitude'].values if x > upper or x < lower]
print ("number of mild outliers:", len(outliers))
#print(lower, upper)

number of mild outliers: 0


#### Before Handling

In [0]:
# sns.boxplot(y=train_df['longitude'])

#### After Handling

In [21]:
whichrow = 0
for row in train_df['longitude']:
    if row>upper or row<lower:
        train_df.loc[train_df.index[whichrow], 'longitude'] = np.nan
        whichrow+=1
    else:
        whichrow += 1
        
mean_val = nanmean(train_df['longitude'])
print(mean_val)

whichrow = 0
for row in train_df['longitude']:
    if math.isnan(row):
        train_df.loc[train_df.index[whichrow],'longitude'] = mean_val
        whichrow+=1
    else:
        whichrow+=1

# sns.boxplot(y=train_df['longitude'] , whis = [0,100])

-73.97370231860559


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: scipy.nanmean is deprecated and will be removed in SciPy 2.0.0, use numpy.nanmean instead
  if __name__ == '__main__':


We found the Tukey inner and outer fences to find the mild and extreme outliers. The outer fences still included listings that were within the NYC borough bounds, accoding to Google Maps. The very bottom tip of New York the island is around -74.01 and -73.88 is in the Bronx. So we decided to only deal with the mild outliers that encompassed this. We replaced with the mean value  of 40.7513098874893 so we have more data points to train with. The fence cutoffs we used were -74.04704999999998 and -73.89945000000003. This dealt with outliers such as listings located in LA because we wanted to keep the listings within NYC. The new minimum and maximum longitude and latitude are now -74.0454 and -73.8995, respectively.

### Bedrooms

In [22]:
q25, q75 = np.percentile(train_df['bedrooms'].values, 25), np.percentile(train_df['bedrooms'].values, 75)
iqr = q75-q25
cutoff = iqr * 1.5
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['bedrooms'].values if x > upper or x < lower]
print ("number of mild outliers:", len(outliers))
print(lower, upper)

number of mild outliers: 2226
-0.5 3.5


In [23]:
q25, q75 = np.percentile(train_df['bedrooms'].values, 25), np.percentile(train_df['bedrooms'].values, 75)
iqr = q75-q25
cutoff = iqr * 3
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['bedrooms'].values if x > upper or x < lower]
print ("number of mild outliers:", len(outliers))
print(lower, upper)

number of mild outliers: 50
-2.0 5.0


#### Before Handling

In [0]:
# sns.boxplot(y=train_df['bedrooms'])

We found the Tukey inner and outer fences to find the mild and extreme outliers. There were 'outliers' but looking at the values, they seem to make sense, given the prices of our listings. It would make sense for a penthouse or large properties to have many bedrooms so a value like 8 isn't an error. We decided to leave this many values unchanged.

### Bathroom

In [25]:
q25, q75 = np.percentile(train_df['bathrooms'].values, 25), np.percentile(train_df['bathrooms'].values, 75)
iqr = q75-q25
cutoff = iqr * 1.5
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['bathrooms'].values if x > upper or x < lower]
print ("number of mild outliers:", len(outliers))
print(lower,upper)

number of mild outliers: 9930
1.0 1.0


In [26]:
q25, q75 = np.percentile(train_df['bathrooms'].values, 25), np.percentile(train_df['bathrooms'].values, 75)
iqr = q75-q25
cutoff = iqr * 3
lower, upper = q25 - cutoff, q75 + cutoff
outliers = [x for x in train_df['bathrooms'].values if x > upper or x < lower]
print ("number of mild outliers:", len(outliers))
print(lower,upper)

number of mild outliers: 9930
1.0 1.0


## Text processing

In [0]:
# convert features list to string
train_df['features'] = train_df['features'].apply(lambda x: ' '.join(x))

In [0]:
vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', token_pattern=r'^[a-zA-Z][a-zA-Z]+')
w = vectorizer.fit_transform(train_df['features']);
tempd = pd.DataFrame(w.toarray(), columns=vectorizer.get_feature_names())
train_df.reset_index(inplace=True, drop=True)
train_df = pd.concat([train_df, tempd], axis=1)

## MILESTONE3

### Importing

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import Perceptron
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model

### Data processing

In [29]:
train_df['interest_level'] = train_df['interest_level'].astype('category')
train_df['target'] = train_df['interest_level'].cat.codes
train_df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level,absolute,actual,air,assigned,available,balcony,bike,brand,building,cable,cats,childrens,close,common,courtyard,deck,deco,decorative,dining,dishwasher,dogs,doorman,duplex,dw,eat,...,laundry,live,loft,low,multi,new,newly,nyu,offering,outdoor,parking,patio,pets,pool,postwar,pre,prewar,private,queen,recreational,reduced,renovated,residents,roof,short,stainless,storage,swimming,terrace,townhouse,triplex,view,views,walk,washer,wifi,wonderful,wood,yard,target
0,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,Dining Room Pre-War Laundry in Building Dishwa...,40.7108,7170325,-73.9539,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/7170325_3bb5ac84...,2400.0,145 Borinquen Place,medium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,Doorman Elevator Laundry in Building Dishwashe...,40.7513,7092344,-73.9722,955db33477af4f40004820b4aed804a0,[https://photos.renthop.com/2/7092344_7663c19a...,3800.0,230 East 44th,low,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,Doorman Elevator Laundry in Building Laundry i...,40.7575,7158677,-73.9625,c8b10a317b766204f08e613cef4ce7a0,[https://photos.renthop.com/2/7158677_c897a134...,3495.0,405 East 56th Street,medium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000.0,792 Metropolitan Avenue,medium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,1.0,0,bfb9405149bfff42a92980b594c28234,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,Doorman Elevator Fitness Center Laundry in Bui...,40.7439,7225292,-73.9743,2c3b41f588fbb5234d8a1e885a436cfa,[https://photos.renthop.com/2/7225292_901f1984...,2795.0,340 East 34th Street,low,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [0]:
feat1 = ['interest_level','building_id','created', 'description','display_address','features', 'listing_id','manager_id','photos', 'street_address', 'target' ]
x = train_df.drop(feat1,axis=1)
y = train_df['target']

In [0]:
test_df = pd.read_json("test.json")
test_df['features'] = test_df['features'].apply(lambda x: ' '.join(x))
w = vectorizer.transform(test_df['features'])
tempd = pd.DataFrame(w.toarray(), columns=vectorizer.get_feature_names())
test_df.reset_index(inplace=True, drop=True)
test_df = pd.concat([test_df, tempd], axis=1)
feat2 = ['building_id','created', 'description','display_address','features', 'listing_id','manager_id','photos', 'street_address']
testing = test_df.drop(feat2, axis=1)

### MLP classifier

#### Starting Version

In [0]:


mlp = MLPClassifier(max_iter=100)
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50)],
    'activation': ['logistic', 'relu'],
    'alpha': [0.0001,0.001,0.01,0.05,0.1,1],
}

from sklearn.model_selection import GridSearchCV

search = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
search.fit(np.array(x), np.array(y))

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=3, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=100, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=None, shuffle=True,
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'

In [0]:
print('Best parameters found:\n', search.best_params_)
means = search.cv_results_['mean_test_score']
stds = search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (50, 100, 50)}
0.701 (+/-0.005) for {'activation': 'logistic', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50)}
0.699 (+/-0.005) for {'activation': 'logistic', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 100, 50)}
0.698 (+/-0.004) for {'activation': 'logistic', 'alpha': 0.001, 'hidden_layer_sizes': (50, 50, 50)}
0.692 (+/-0.029) for {'activation': 'logistic', 'alpha': 0.001, 'hidden_layer_sizes': (50, 100, 50)}
0.697 (+/-0.002) for {'activation': 'logistic', 'alpha': 0.01, 'hidden_layer_sizes': (50, 50, 50)}
0.698 (+/-0.001) for {'activation': 'logistic', 'alpha': 0.01, 'hidden_layer_sizes': (50, 100, 50)}
0.697 (+/-0.003) for {'activation': 'logistic', 'alpha': 0.05, 'hidden_layer_sizes': (50, 50, 50)}
0.699 (+/-0.004) for {'activation': 'logistic', 'alpha': 0.05, 'hidden_layer_sizes': (50, 100, 50)}
0.695 (+/-0.000) for {'activation': 'logistic', 'alpha': 0.1, 'hidden_layer_sizes': (50, 50, 50)}


In [83]:
## cross validation

train_df_cv = shuffle(train_df)
train_df_cv.reset_index(inplace=True, drop=True)
x_cv = train_df_cv.drop(feat1, axis=1)
y_cv = train_df_cv['target']


kf = KFold(n_splits=5)
i=1
tsum = 0
for train_index, test_index in kf.split(x_cv):
    x_train, x_test = x_cv.loc[train_index[0]:train_index[-1]], x_cv.loc[test_index[0]:test_index[-1]]
    y_train, y_test = y_cv.loc[train_index[0]:train_index[-1]], y_cv.loc[test_index[0]:test_index[-1]]
    clf = MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=100)
    clf.fit(np.array(x_train),np.array(y_train))
    y_pred = clf.predict(x_test)
    y_pred2 = clf.predict_proba(x_test)
    
    print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
    print(classification_report(y_test,y_pred))
    print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
    i=i+1
    tsum = tsum + log_loss(y_test,y_pred2)
    
print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")

log_loss for  1  :  0.7089644728123278 

              precision    recall  f1-score   support

           0       0.54      0.18      0.27       761
           1       0.72      0.99      0.83      6849
           2       0.45      0.03      0.06      2261

    accuracy                           0.71      9871
   macro avg       0.57      0.40      0.39      9871
weighted avg       0.64      0.71      0.61      9871

SVM Accuracy Score ->  70.67166447168474 for  1 

log_loss for  2  :  0.7127078534402234 

              precision    recall  f1-score   support

           0       0.53      0.17      0.26       775
           1       0.72      0.96      0.82      6793
           2       0.39      0.09      0.15      2303

    accuracy                           0.70      9871
   macro avg       0.55      0.41      0.41      9871
weighted avg       0.63      0.70      0.62      9871

SVM Accuracy Score ->  69.72951068787356 for  2 

log_loss for  3  :  0.7028847996587603 

              p

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


log_loss for  4  :  0.7261224522657062 

              precision    recall  f1-score   support

           0       0.60      0.06      0.11       768
           1       0.70      1.00      0.82      6844
           2       0.44      0.02      0.03      2258

    accuracy                           0.70      9870
   macro avg       0.58      0.36      0.32      9870
weighted avg       0.63      0.70      0.59      9870

SVM Accuracy Score ->  69.95947315096251 for  4 

log_loss for  5  :  0.7338421538375419 

              precision    recall  f1-score   support

           0       0.36      0.19      0.25       750
           1       0.74      0.92      0.82      6887
           2       0.36      0.14      0.20      2233

    accuracy                           0.69      9870
   macro avg       0.49      0.42      0.42      9870
weighted avg       0.62      0.69      0.64      9870

SVM Accuracy Score ->  68.9564336372847 for  5 

log_loss average evaluation results of 5 classifiers:  0.

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [0]:
prediction = search.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_search.csv', index=False) 

#### increase iteration

In [0]:
mlp_s1 = MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=1000)
mlp_s1.fit(np.array(x),np.array(y))

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50, 100, 50), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
## cross validation

train_df_cv = shuffle(train_df)
train_df_cv.reset_index(inplace=True, drop=True)
x_cv = train_df_cv.drop(feat1, axis=1)
y_cv = train_df_cv['target']


kf = KFold(n_splits=5)
i=1
tsum = 0
for train_index, test_index in kf.split(x_cv):
    x_train, x_test = x_cv.loc[train_index[0]:train_index[-1]], x_cv.loc[test_index[0]:test_index[-1]]
    y_train, y_test = y_cv.loc[train_index[0]:train_index[-1]], y_cv.loc[test_index[0]:test_index[-1]]
    clf = MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=1000)
    clf.fit(np.array(x_train),np.array(y_train))
    y_pred = clf.predict(x_test)
    y_pred2 = clf.predict_proba(x_test)
    
    print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
    print(classification_report(y_test,y_pred))
    print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
    i=i+1
    tsum = tsum + log_loss(y_test,y_pred2)
    
print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")

log_loss for  1  :  0.7139833833373197 

              precision    recall  f1-score   support

           0       0.62      0.10      0.17       790
           1       0.72      0.97      0.83      6816
           2       0.42      0.11      0.17      2265

    accuracy                           0.70      9871
   macro avg       0.59      0.39      0.39      9871
weighted avg       0.65      0.70      0.62      9871

SVM Accuracy Score ->  70.41839732549894 for  1 

log_loss for  2  :  0.7455266377583426 

              precision    recall  f1-score   support

           0       0.39      0.23      0.29       732
           1       0.75      0.91      0.82      6922
           2       0.37      0.17      0.24      2217

    accuracy                           0.69      9871
   macro avg       0.50      0.44      0.45      9871
weighted avg       0.64      0.69      0.65      9871

SVM Accuracy Score ->  69.48637422753521 for  2 

log_loss for  3  :  0.7320912405528235 

              p

In [0]:
prediction = mlp_s1.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_search2.csv', index=False) 

### Stacking 

#### Randomforest classifier

In [0]:
rf = RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50)
rf.fit(np.array(x),np.array(y))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
## cross validation

train_df_cv = shuffle(train_df)
train_df_cv.reset_index(inplace=True, drop=True)
x_cv = train_df_cv.drop(feat1, axis=1)
y_cv = train_df_cv['target']


kf = KFold(n_splits=5)
i=1
tsum = 0
for train_index, test_index in kf.split(x_cv):
    x_train, x_test = x_cv.loc[train_index[0]:train_index[-1]], x_cv.loc[test_index[0]:test_index[-1]]
    y_train, y_test = y_cv.loc[train_index[0]:train_index[-1]], y_cv.loc[test_index[0]:test_index[-1]]
    clf = RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50)
    clf.fit(np.array(x_train),np.array(y_train))
    y_pred = clf.predict(x_test)
    y_pred2 = clf.predict_proba(x_test)
    
    print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
    print(classification_report(y_test,y_pred))
    print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
    i=i+1
    tsum = tsum + log_loss(y_test,y_pred2)
    
print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")

log_loss for  1  :  0.694029277107931 

              precision    recall  f1-score   support

           0       0.72      0.02      0.04       796
           1       0.70      1.00      0.82      6841
           2       0.41      0.01      0.01      2234

    accuracy                           0.70      9871
   macro avg       0.61      0.34      0.29      9871
weighted avg       0.63      0.70      0.58      9871

SVM Accuracy Score ->  69.52689697092494 for  1 

log_loss for  2  :  0.6617035987967547 

              precision    recall  f1-score   support

           0       0.60      0.00      0.01       740
           1       0.70      1.00      0.82      6906
           2       0.43      0.00      0.01      2225

    accuracy                           0.70      9871
   macro avg       0.58      0.34      0.28      9871
weighted avg       0.63      0.70      0.58      9871

SVM Accuracy Score ->  70.04356194914396 for  2 

log_loss for  3  :  0.6761706375041623 

              pr

In [0]:
prediction = rf.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_rf.csv', index=False) 

#### added randomforest

In [0]:
est = [
       ('mlp3', MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=1000)),
       ('rf', RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50))
]

mlp_rf1 = StackingClassifier(estimators=est, final_estimator=RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50))
mlp_rf1.fit(np.array(x),np.array(y))

StackingClassifier(cv=None,
                   estimators=[('mlp3',
                                MLPClassifier(activation='relu', alpha=0.001,
                                              batch_size='auto', beta_1=0.9,
                                              beta_2=0.999,
                                              early_stopping=False,
                                              epsilon=1e-08,
                                              hidden_layer_sizes=(50, 100, 50),
                                              learning_rate='constant',
                                              learning_rate_init=0.001,
                                              max_fun=15000, max_iter=1000,
                                              momentum=0.9, n_iter_no_change=10,
                                              nesterovs_momentum=True,
                                              power_t=0.5, rand...
                                                          criterion='

In [0]:
## cross validation

train_df_cv = shuffle(train_df)
train_df_cv.reset_index(inplace=True, drop=True)
x_cv = train_df_cv.drop(feat1, axis=1)
y_cv = train_df_cv['target']


kf = KFold(n_splits=5)
i=1
tsum = 0
for train_index, test_index in kf.split(x_cv):
    x_train, x_test = x_cv.loc[train_index[0]:train_index[-1]], x_cv.loc[test_index[0]:test_index[-1]]
    y_train, y_test = y_cv.loc[train_index[0]:train_index[-1]], y_cv.loc[test_index[0]:test_index[-1]]
    clf = StackingClassifier(estimators=est, final_estimator=RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50))
    clf.fit(np.array(x_train),np.array(y_train))
    y_pred = clf.predict(x_test)
    y_pred2 = clf.predict_proba(x_test)
    
    print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
    print(classification_report(y_test,y_pred))
    print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
    i=i+1
    tsum = tsum + log_loss(y_test,y_pred2)
    
print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")

log_loss for  1  :  0.671299841260537 

              precision    recall  f1-score   support

           0       0.52      0.18      0.27       730
           1       0.75      0.93      0.84      6912
           2       0.44      0.21      0.29      2229

    accuracy                           0.72      9871
   macro avg       0.57      0.44      0.46      9871
weighted avg       0.67      0.72      0.67      9871

SVM Accuracy Score ->  71.53277276871644 for  1 

log_loss for  2  :  0.6381761094052887 

              precision    recall  f1-score   support

           0       0.53      0.16      0.24       788
           1       0.77      0.93      0.84      6871
           2       0.46      0.28      0.35      2212

    accuracy                           0.72      9871
   macro avg       0.58      0.46      0.48      9871
weighted avg       0.68      0.72      0.68      9871

SVM Accuracy Score ->  72.10009117617263 for  2 

log_loss for  3  :  0.6543854768662569 

              pr

In [0]:
prediction = mlp_rf1.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_mlp+rf1.csv', index=False) 

In [0]:
est = [
       ('mlp3', MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=1000)),
       ('rf', RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50))
]

mlp_rf2 = StackingClassifier(estimators=est, final_estimator=MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=1000))
mlp_rf2.fit(np.array(x),np.array(y))

StackingClassifier(cv=None,
                   estimators=[('mlp3',
                                MLPClassifier(activation='relu', alpha=0.001,
                                              batch_size='auto', beta_1=0.9,
                                              beta_2=0.999,
                                              early_stopping=False,
                                              epsilon=1e-08,
                                              hidden_layer_sizes=(50, 100, 50),
                                              learning_rate='constant',
                                              learning_rate_init=0.001,
                                              max_fun=15000, max_iter=1000,
                                              momentum=0.9, n_iter_no_change=10,
                                              nesterovs_momentum=True,
                                              power_t=0.5, rand...
                                                 hidden_layer_sizes=(

In [0]:
## cross validation

train_df_cv = shuffle(train_df)
train_df_cv.reset_index(inplace=True, drop=True)
x_cv = train_df_cv.drop(feat1, axis=1)
y_cv = train_df_cv['target']


kf = KFold(n_splits=5)
i=1
tsum = 0
for train_index, test_index in kf.split(x_cv):
    x_train, x_test = x_cv.loc[train_index[0]:train_index[-1]], x_cv.loc[test_index[0]:test_index[-1]]
    y_train, y_test = y_cv.loc[train_index[0]:train_index[-1]], y_cv.loc[test_index[0]:test_index[-1]]
    clf = StackingClassifier(estimators=est, final_estimator=MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=1000))
    clf.fit(np.array(x_train),np.array(y_train))
    y_pred = clf.predict(x_test)
    y_pred2 = clf.predict_proba(x_test)
    
    print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
    print(classification_report(y_test,y_pred))
    print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
    i=i+1
    tsum = tsum + log_loss(y_test,y_pred2)
    
print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")

log_loss for  1  :  0.6763492189847757 

              precision    recall  f1-score   support

           0       0.45      0.21      0.29       776
           1       0.76      0.93      0.83      6840
           2       0.44      0.21      0.29      2255

    accuracy                           0.71      9871
   macro avg       0.55      0.45      0.47      9871
weighted avg       0.66      0.71      0.67      9871

SVM Accuracy Score ->  70.98571573295513 for  1 

log_loss for  2  :  0.6367575361451367 

              precision    recall  f1-score   support

           0       0.61      0.13      0.21       733
           1       0.75      0.96      0.84      6863
           2       0.51      0.20      0.29      2275

    accuracy                           0.72      9871
   macro avg       0.62      0.43      0.45      9871
weighted avg       0.68      0.72      0.67      9871

SVM Accuracy Score ->  72.35335832235842 for  2 

log_loss for  3  :  0.6369267771079636 

              p

In [0]:
prediction = mlp_rf2.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_mlp+rf2.csv', index=False) 

#### KNeighbors classifier

In [75]:
neigh = KNeighborsClassifier(n_neighbors= 150, leaf_size = 100, algorithm = 'ball_tree')
neigh.fit(np.array(x), np.array(y))

# neigh_base = KNeighborsClassifier()

# parameter_space = {
#     'n_neighbors': [20, 40, 60],
#     # 'weights': ['uniform', 'distance'],
#     'algorithm': ['ball_tree', 'kd_tree'],
#     # 'leaf_size': [20,30, 40]
#     'leaf_size': [50, 100, 150, 200]
# }

# from sklearn.model_selection import GridSearchCV

# neigh = GridSearchCV(neigh_base, parameter_space, n_jobs=-1, cv=3)
# neigh.fit(np.array(x), np.array(y))

KNeighborsClassifier(algorithm='ball_tree', leaf_size=100, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=150, p=2,
                     weights='uniform')

In [61]:
# print('Best parameters found:\n', neigh.best_params_)
# means = neigh.cv_results_['mean_test_score']
# stds = neigh.cv_results_['std_test_score']
# for mean, std, params in zip(means, stds, neigh.cv_results_['params']):
#     print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'algorithm': 'ball_tree', 'leaf_size': 100, 'n_neighbors': 40}
0.698 (+/-0.002) for {'algorithm': 'ball_tree', 'leaf_size': 50, 'n_neighbors': 20}
0.701 (+/-0.003) for {'algorithm': 'ball_tree', 'leaf_size': 50, 'n_neighbors': 40}
0.700 (+/-0.001) for {'algorithm': 'ball_tree', 'leaf_size': 50, 'n_neighbors': 60}
0.698 (+/-0.002) for {'algorithm': 'ball_tree', 'leaf_size': 100, 'n_neighbors': 20}
0.701 (+/-0.003) for {'algorithm': 'ball_tree', 'leaf_size': 100, 'n_neighbors': 40}
0.700 (+/-0.001) for {'algorithm': 'ball_tree', 'leaf_size': 100, 'n_neighbors': 60}
0.698 (+/-0.002) for {'algorithm': 'ball_tree', 'leaf_size': 150, 'n_neighbors': 20}
0.700 (+/-0.003) for {'algorithm': 'ball_tree', 'leaf_size': 150, 'n_neighbors': 40}
0.700 (+/-0.001) for {'algorithm': 'ball_tree', 'leaf_size': 150, 'n_neighbors': 60}
0.698 (+/-0.002) for {'algorithm': 'ball_tree', 'leaf_size': 200, 'n_neighbors': 20}
0.700 (+/-0.003) for {'algorithm': 'ball_tree', 'leaf_size': 200,

In [76]:
## cross validation

train_df_cv = shuffle(train_df)
train_df_cv.reset_index(inplace=True, drop=True)
x_cv = train_df_cv.drop(feat1, axis=1)
y_cv = train_df_cv['target']


kf = KFold(n_splits=5)
i=1
tsum = 0
for train_index, test_index in kf.split(x_cv):
    x_train, x_test = x_cv.loc[train_index[0]:train_index[-1]], x_cv.loc[test_index[0]:test_index[-1]]
    y_train, y_test = y_cv.loc[train_index[0]:train_index[-1]], y_cv.loc[test_index[0]:test_index[-1]]
    clf = KNeighborsClassifier(n_neighbors= 150, leaf_size = 100, algorithm = 'ball_tree')
    clf.fit(np.array(x_train),np.array(y_train))
    y_pred = clf.predict(x_test)
    y_pred2 = clf.predict_proba(x_test)
    
    print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
    print(classification_report(y_test,y_pred))
    print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
    i=i+1
    tsum = tsum + log_loss(y_test,y_pred2)
    
print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")

log_loss for  1  :  0.7268914942734905 

              precision    recall  f1-score   support

           0       0.51      0.10      0.16       790
           1       0.70      0.99      0.82      6828
           2       0.41      0.02      0.03      2253

    accuracy                           0.70      9871
   macro avg       0.54      0.37      0.34      9871
weighted avg       0.62      0.70      0.59      9871

SVM Accuracy Score ->  69.56741971431467 for  1 

log_loss for  2  :  0.7143553646699005 

              precision    recall  f1-score   support

           0       0.55      0.11      0.19       765
           1       0.71      0.99      0.82      6847
           2       0.41      0.02      0.05      2259

    accuracy                           0.70      9871
   macro avg       0.56      0.37      0.35      9871
weighted avg       0.63      0.70      0.60      9871

SVM Accuracy Score ->  69.88147097558505 for  2 

log_loss for  3  :  0.6979623749219386 

              p

In [0]:
prediction = neigh.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_neigh.csv', index=False) 

#### added KNeighbors

In [78]:
est = [
       ('mlp3', MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=1000)),
       ('rf', RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50)),
       ('knn', KNeighborsClassifier(n_neighbors= 150, leaf_size = 100, algorithm = 'ball_tree'))
]

mlp_rf2_knn = StackingClassifier(estimators=est, final_estimator=MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=1000))
mlp_rf2_knn.fit(np.array(x),np.array(y))

StackingClassifier(cv=None,
                   estimators=[('mlp3',
                                MLPClassifier(activation='relu', alpha=0.001,
                                              batch_size='auto', beta_1=0.9,
                                              beta_2=0.999,
                                              early_stopping=False,
                                              epsilon=1e-08,
                                              hidden_layer_sizes=(50, 100, 50),
                                              learning_rate='constant',
                                              learning_rate_init=0.001,
                                              max_fun=15000, max_iter=1000,
                                              momentum=0.9, n_iter_no_change=10,
                                              nesterovs_momentum=True,
                                              power_t=0.5, rand...
                                                 hidden_layer_sizes=(

In [80]:
## cross validation

train_df_cv = shuffle(train_df)
train_df_cv.reset_index(inplace=True, drop=True)
x_cv = train_df_cv.drop(feat1, axis=1)
y_cv = train_df_cv['target']


kf = KFold(n_splits=5)
i=1
tsum = 0
for train_index, test_index in kf.split(x_cv):
    x_train, x_test = x_cv.loc[train_index[0]:train_index[-1]], x_cv.loc[test_index[0]:test_index[-1]]
    y_train, y_test = y_cv.loc[train_index[0]:train_index[-1]], y_cv.loc[test_index[0]:test_index[-1]]
    clf = StackingClassifier(estimators=est, final_estimator=MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=1000))
    clf.fit(np.array(x_train),np.array(y_train))
    y_pred = clf.predict(x_test)
    y_pred2 = clf.predict_proba(x_test)
    
    print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
    print(classification_report(y_test,y_pred))
    print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
    i=i+1
    tsum = tsum + log_loss(y_test,y_pred2)
    
print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")

log_loss for  1  :  0.6715647450204595 

              precision    recall  f1-score   support

           0       0.48      0.12      0.19       767
           1       0.74      0.95      0.83      6854
           2       0.42      0.16      0.24      2250

    accuracy                           0.71      9871
   macro avg       0.55      0.41      0.42      9871
weighted avg       0.65      0.71      0.65      9871

SVM Accuracy Score ->  70.81349407354878 for  1 

log_loss for  2  :  0.6381141093903572 

              precision    recall  f1-score   support

           0       0.56      0.14      0.22       778
           1       0.76      0.94      0.84      6851
           2       0.45      0.25      0.32      2242

    accuracy                           0.72      9871
   macro avg       0.59      0.44      0.46      9871
weighted avg       0.67      0.72      0.67      9871

SVM Accuracy Score ->  71.78603991490225 for  2 

log_loss for  3  :  0.6234063440264569 

              p

In [0]:
prediction = mlp_rf2_knn.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_mlp+rf2+knn.csv', index=False) 

### reduce overfitting

In [105]:
est = [
       ('mlp3', MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=500)),
       ('rf', RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50)),
       ('knn', KNeighborsClassifier(n_neighbors= 150, leaf_size = 100, algorithm = 'ball_tree'))
]

mlp_rf2_knn2 = StackingClassifier(estimators=est, final_estimator=MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=500))
mlp_rf2_knn2.fit(np.array(x),np.array(y))

StackingClassifier(cv=None,
                   estimators=[('mlp3',
                                MLPClassifier(activation='relu', alpha=0.001,
                                              batch_size='auto', beta_1=0.9,
                                              beta_2=0.999,
                                              early_stopping=False,
                                              epsilon=1e-08,
                                              hidden_layer_sizes=(50, 100, 50),
                                              learning_rate='constant',
                                              learning_rate_init=0.001,
                                              max_fun=15000, max_iter=500,
                                              momentum=0.9, n_iter_no_change=10,
                                              nesterovs_momentum=True,
                                              power_t=0.5, rando...
                                                 epsilon=1e-08,
     

In [111]:
## cross validation

train_df_cv = shuffle(train_df)
train_df_cv.reset_index(inplace=True, drop=True)
x_cv = train_df_cv.drop(feat1, axis=1)
y_cv = train_df_cv['target']

kf = KFold(n_splits=5)
i=1
tsum = 0
for train_index, test_index in kf.split(x_cv):
    x_train, x_test = x_cv.loc[train_index[0]:train_index[-1]], x_cv.loc[test_index[0]:test_index[-1]]
    y_train, y_test = y_cv.loc[train_index[0]:train_index[-1]], y_cv.loc[test_index[0]:test_index[-1]]
    clf = StackingClassifier(estimators=est, final_estimator=MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=500))
    clf.fit(np.array(x_train),np.array(y_train))
    y_pred = clf.predict(x_test)
    y_pred2 = clf.predict_proba(x_test)
    
    print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
    print(classification_report(y_test,y_pred))
    print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
    i=i+1
    tsum = tsum + log_loss(y_test,y_pred2)
    
print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")

log_loss for  1  :  0.6868556265995636 

              precision    recall  f1-score   support

           0       0.57      0.14      0.23       799
           1       0.72      0.96      0.83      6750
           2       0.41      0.12      0.18      2322

    accuracy                           0.70      9871
   macro avg       0.57      0.41      0.41      9871
weighted avg       0.64      0.70      0.63      9871

SVM Accuracy Score ->  69.78016411711073 for  1 

log_loss for  2  :  0.6272705909148935 

              precision    recall  f1-score   support

           0       0.54      0.16      0.25       732
           1       0.75      0.96      0.84      6840
           2       0.50      0.19      0.27      2299

    accuracy                           0.72      9871
   macro avg       0.59      0.44      0.45      9871
weighted avg       0.67      0.72      0.67      9871

SVM Accuracy Score ->  72.19126734879951 for  2 

log_loss for  3  :  0.6275296627460556 

              p

In [0]:
prediction = mlp_rf2_knn2.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_mlp+rf2+knn2.csv', index=False) 

## Two layer stacking (no improvement)

In [0]:
layer1_est = [
              ('mlp3', MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=1000)),
              ('rf', RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50))
]

layer2_est = [
              ('mlp3', MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=1000)),
              ('rf', RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50))
]

layer2 = StackingClassifier(estimators=layer2_est, final_estimator=MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=1000))

twolayer = StackingClassifier(estimators=layer1_est, final_estimator=layer2)
twolayer.fit(np.array(x), np.array(y))



StackingClassifier(cv=None,
                   estimators=[('mlp3',
                                MLPClassifier(activation='relu', alpha=0.001,
                                              batch_size='auto', beta_1=0.9,
                                              beta_2=0.999,
                                              early_stopping=False,
                                              epsilon=1e-08,
                                              hidden_layer_sizes=(50, 100, 50),
                                              learning_rate='constant',
                                              learning_rate_init=0.001,
                                              max_fun=15000, max_iter=1000,
                                              momentum=0.9, n_iter_no_change=10,
                                              nesterovs_momentum=True,
                                              power_t=0.5, rand...
                                                                     

In [0]:
prediction = twolayer.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_twolayer.csv', index=False) 

In [0]:
layer1_est = [
              ('rf1', RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50)),
              ('rf2', RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=25,n_estimators=55)),
              ('rf3', RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=30,n_estimators=60))
]

layer2_est = [
              ('mlp3', MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=1000)),
              ('rf', RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50))
]

layer2 = StackingClassifier(estimators=layer2_est, final_estimator=MLPClassifier(alpha = 0.001, hidden_layer_sizes = (50,100,50), max_iter=1000))

twolayer2 = StackingClassifier(estimators=layer1_est, final_estimator=layer2)
twolayer2.fit(np.array(x), np.array(y))

StackingClassifier(cv=None,
                   estimators=[('rf1',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=20,
                                                       min_samples_split=2,
                                   

In [0]:
prediction = twolayer2.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_twolayer2.csv', index=False) 

## Other attempts


In [0]:
mlp1 = MLPClassifier(alpha = 0.1, hidden_layer_sizes = (50,50,50), max_iter=1000)
mlp1.fit(np.array(x),np.array(y))

MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50, 50, 50), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
## cross validation

train_df_cv = shuffle(train_df)
train_df_cv.reset_index(inplace=True, drop=True)
x_cv = train_df_cv.drop(feat1, axis=1)
y_cv = train_df_cv['target']


kf = KFold(n_splits=5)
i=1
tsum = 0
for train_index, test_index in kf.split(x):
    x_train, x_test = x_cv.loc[train_index[0]:train_index[-1]], x_cv.loc[test_index[0]:test_index[-1]]
    y_train, y_test = y_cv.loc[train_index[0]:train_index[-1]], y_cv.loc[test_index[0]:test_index[-1]]
    clf = MLPClassifier(alpha = 0.1, hidden_layer_sizes = (50,50,50), max_iter=1000)
    clf.fit(np.array(x_train),np.array(y_train))
    y_pred = clf.predict(x_test)
    y_pred2 = clf.predict_proba(x_test)
    
    print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
    print(classification_report(y_test,y_pred))
    print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
    i=i+1
    tsum = tsum + log_loss(y_test,y_pred2)
    
print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")

log_loss for  1  :  0.7444403180702548 

              precision    recall  f1-score   support

           0       0.41      0.19      0.26       785
           1       0.71      0.99      0.82      6816
           2       0.55      0.01      0.01      2270

    accuracy                           0.70      9871
   macro avg       0.56      0.39      0.37      9871
weighted avg       0.65      0.70      0.59      9871

SVM Accuracy Score ->  69.7092493161787 for  1 

log_loss for  2  :  0.724957984903575 

              precision    recall  f1-score   support

           0       0.60      0.11      0.18       769
           1       0.70      1.00      0.82      6847
           2       0.53      0.01      0.03      2255

    accuracy                           0.70      9871
   macro avg       0.61      0.37      0.34      9871
weighted avg       0.66      0.70      0.59      9871

SVM Accuracy Score ->  70.17526086516057 for  2 

log_loss for  3  :  0.7618571647416151 

              pre

In [0]:
# testing = test_df.drop(feat2, axis=1)
# prediction = svc_class.predict_proba(testing)
# result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
# result.to_csv('m3_1.csv', index=False) 

prediction = mlp1.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_1.csv', index=False) 

In [0]:
mlp2 = MLPClassifier(alpha = 0.1, hidden_layer_sizes = (50,50,50), max_iter=2000)
mlp2.fit(np.array(x),np.array(y))

MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50, 50, 50), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=2000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
## cross validation

train_df_cv = shuffle(train_df)
train_df_cv.reset_index(inplace=True, drop=True)
x_cv = train_df_cv.drop(feat1, axis=1)
y_cv = train_df_cv['target']


kf = KFold(n_splits=5)
i=1
tsum = 0
for train_index, test_index in kf.split(x):
    x_train, x_test = x_cv.loc[train_index[0]:train_index[-1]], x_cv.loc[test_index[0]:test_index[-1]]
    y_train, y_test = y_cv.loc[train_index[0]:train_index[-1]], y_cv.loc[test_index[0]:test_index[-1]]
    clf = MLPClassifier(alpha = 0.1, hidden_layer_sizes = (50,50,50), max_iter=2000)
    clf.fit(np.array(x_train),np.array(y_train))
    y_pred = clf.predict(x_test)
    y_pred2 = clf.predict_proba(x_test)
    
    print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
    print(classification_report(y_test,y_pred))
    print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
    i=i+1
    tsum = tsum + log_loss(y_test,y_pred2)
    
print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")

log_loss for  1  :  0.7428073507641397 

              precision    recall  f1-score   support

           0       0.62      0.09      0.16       758
           1       0.70      1.00      0.82      6826
           2       0.47      0.01      0.02      2287

    accuracy                           0.70      9871
   macro avg       0.60      0.37      0.34      9871
weighted avg       0.64      0.70      0.59      9871

SVM Accuracy Score ->  69.85107891804276 for  1 

log_loss for  2  :  0.7396339186924779 

              precision    recall  f1-score   support

           0       0.68      0.09      0.16       802
           1       0.71      1.00      0.83      6917
           2       0.00      0.00      0.00      2152

    accuracy                           0.71      9871
   macro avg       0.46      0.36      0.33      9871
weighted avg       0.55      0.71      0.59      9871

SVM Accuracy Score ->  70.631141728295 for  2 

log_loss for  3  :  0.7630944597011607 

              pre

In [0]:
prediction = mlp2.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_2.csv', index=False) 

In [0]:
mlp3 = MLPClassifier(alpha = 0.01, hidden_layer_sizes = (50,50,50), max_iter=2000)
mlp3.fit(np.array(x),np.array(y))

MLPClassifier(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50, 50, 50), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=2000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
## cross validation

train_df_cv = shuffle(train_df)
train_df_cv.reset_index(inplace=True, drop=True)
x_cv = train_df_cv.drop(feat1, axis=1)
y_cv = train_df_cv['target']


kf = KFold(n_splits=5)
i=1
tsum = 0
for train_index, test_index in kf.split(x):
    x_train, x_test = x_cv.loc[train_index[0]:train_index[-1]], x_cv.loc[test_index[0]:test_index[-1]]
    y_train, y_test = y_cv.loc[train_index[0]:train_index[-1]], y_cv.loc[test_index[0]:test_index[-1]]
    clf = MLPClassifier(alpha = 0.01, hidden_layer_sizes = (50,50,50), max_iter=2000)
    clf.fit(np.array(x_train),np.array(y_train))
    y_pred = clf.predict(x_test)
    y_pred2 = clf.predict_proba(x_test)
    
    print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
    print(classification_report(y_test,y_pred))
    print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
    i=i+1
    tsum = tsum + log_loss(y_test,y_pred2)
    
print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")

log_loss for  1  :  0.7173075994558318 

              precision    recall  f1-score   support

           0       0.57      0.11      0.19       811
           1       0.72      0.99      0.83      6919
           2       0.30      0.02      0.03      2141

    accuracy                           0.71      9871
   macro avg       0.53      0.37      0.35      9871
weighted avg       0.61      0.71      0.61      9871

SVM Accuracy Score ->  70.81349407354878 for  1 

log_loss for  2  :  0.72739583797117 

              precision    recall  f1-score   support

           0       0.48      0.16      0.24       764
           1       0.71      0.98      0.83      6815
           2       0.39      0.04      0.08      2292

    accuracy                           0.70      9871
   macro avg       0.53      0.39      0.38      9871
weighted avg       0.62      0.70      0.61      9871

SVM Accuracy Score ->  70.01316989160166 for  2 

log_loss for  3  :  0.7707235332404987 

              pre

In [0]:
prediction = mlp3.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_3.csv', index=False) 

In [0]:
est = [
       ('mlp3', MLPClassifier(alpha = 0.01, hidden_layer_sizes = (50,50,50), max_iter=2000)),
       ('rf', RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50))
]

mlp_stack = StackingClassifier(estimators=est, final_estimator=RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50))
mlp_stack.fit(np.array(x),np.array(y))

StackingClassifier(cv=None,
                   estimators=[('mlp3',
                                MLPClassifier(activation='relu', alpha=0.01,
                                              batch_size='auto', beta_1=0.9,
                                              beta_2=0.999,
                                              early_stopping=False,
                                              epsilon=1e-08,
                                              hidden_layer_sizes=(50, 50, 50),
                                              learning_rate='constant',
                                              learning_rate_init=0.001,
                                              max_fun=15000, max_iter=2000,
                                              momentum=0.9, n_iter_no_change=10,
                                              nesterovs_momentum=True,
                                              power_t=0.5, random...
                                                          criterion='

In [0]:
## cross validation

train_df_cv = shuffle(train_df)
train_df_cv.reset_index(inplace=True, drop=True)
x_cv = train_df_cv.drop(feat1, axis=1)
y_cv = train_df_cv['target']


kf = KFold(n_splits=5)
i=1
tsum = 0
for train_index, test_index in kf.split(x):
    x_train, x_test = x_cv.loc[train_index[0]:train_index[-1]], x_cv.loc[test_index[0]:test_index[-1]]
    y_train, y_test = y_cv.loc[train_index[0]:train_index[-1]], y_cv.loc[test_index[0]:test_index[-1]]
    clf = StackingClassifier(estimators=est, final_estimator=RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50))
    clf.fit(np.array(x),np.array(y))
    y_pred = clf.predict(x_test)
    y_pred2 = clf.predict_proba(x_test)
    
    print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
    print(classification_report(y_test,y_pred))
    print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
    i=i+1
    tsum = tsum + log_loss(y_test,y_pred2)
    
print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")

log_loss for  1  :  0.6297635974951408 

              precision    recall  f1-score   support

           0       0.57      0.13      0.21       749
           1       0.76      0.94      0.84      6903
           2       0.48      0.25      0.33      2219

    accuracy                           0.73      9871
   macro avg       0.60      0.44      0.46      9871
weighted avg       0.68      0.73      0.68      9871

SVM Accuracy Score ->  72.5053186100699 for  1 

log_loss for  2  :  0.6387197582863193 

              precision    recall  f1-score   support

           0       0.63      0.15      0.25       794
           1       0.77      0.93      0.84      6859
           2       0.45      0.29      0.35      2218

    accuracy                           0.72      9871
   macro avg       0.62      0.46      0.48      9871
weighted avg       0.69      0.72      0.69      9871

SVM Accuracy Score ->  72.23179009218924 for  2 

log_loss for  3  :  0.6395917490662623 

              pr

In [0]:
prediction = mlp_stack.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_stack.csv', index=False) 

In [0]:
est = [
       ('mlp3', MLPClassifier(alpha = 0.01, hidden_layer_sizes = (50,50,50), max_iter=2000)),
       ('lr', RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50))
]

mlp_stack2 = StackingClassifier(estimators=est, final_estimator=MLPClassifier(alpha = 0.01, hidden_layer_sizes = (50,50,50), max_iter=2000))
mlp_stack2.fit(np.array(x),np.array(y))

StackingClassifier(cv=None,
                   estimators=[('mlp3',
                                MLPClassifier(activation='relu', alpha=0.01,
                                              batch_size='auto', beta_1=0.9,
                                              beta_2=0.999,
                                              early_stopping=False,
                                              epsilon=1e-08,
                                              hidden_layer_sizes=(50, 50, 50),
                                              learning_rate='constant',
                                              learning_rate_init=0.001,
                                              max_fun=15000, max_iter=2000,
                                              momentum=0.9, n_iter_no_change=10,
                                              nesterovs_momentum=True,
                                              power_t=0.5, random...
                                                 epsilon=1e-08,
     

In [0]:
## cross validation

train_df_cv = shuffle(train_df)
train_df_cv.reset_index(inplace=True, drop=True)
x_cv = train_df_cv.drop(feat1, axis=1)
y_cv = train_df_cv['target']


kf = KFold(n_splits=5)
i=1
tsum = 0
for train_index, test_index in kf.split(x):
    x_train, x_test = x_cv.loc[train_index[0]:train_index[-1]], x_cv.loc[test_index[0]:test_index[-1]]
    y_train, y_test = y_cv.loc[train_index[0]:train_index[-1]], y_cv.loc[test_index[0]:test_index[-1]]
    clf = StackingClassifier(estimators=est, final_estimator=MLPClassifier(alpha = 0.01, hidden_layer_sizes = (50,50,50), max_iter=2000))
    clf.fit(np.array(x),np.array(y))
    y_pred = clf.predict(x_test)
    y_pred2 = clf.predict_proba(x_test)
    
    print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
    print(classification_report(y_test,y_pred))
    print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
    i=i+1
    tsum = tsum + log_loss(y_test,y_pred2)
    
print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")

log_loss for  1  :  0.6298123595318583 

              precision    recall  f1-score   support

           0       0.62      0.20      0.30       791
           1       0.78      0.92      0.85      6856
           2       0.48      0.33      0.39      2224

    accuracy                           0.73      9871
   macro avg       0.63      0.48      0.51      9871
weighted avg       0.70      0.73      0.70      9871

SVM Accuracy Score ->  73.06250633167866 for  1 

log_loss for  2  :  0.6255790120317299 

              precision    recall  f1-score   support

           0       0.49      0.19      0.28       755
           1       0.77      0.94      0.85      6885
           2       0.51      0.28      0.36      2231

    accuracy                           0.73      9871
   macro avg       0.59      0.47      0.50      9871
weighted avg       0.69      0.73      0.69      9871

SVM Accuracy Score ->  73.22459730523757 for  2 

log_loss for  3  :  0.6247915215189076 

              p

In [0]:
prediction = mlp_stack2.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_stack2.csv', index=False) 

In [0]:
est = [
       ('rf1', RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50)),
       ('rf2', RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=25,n_estimators=55)),
       ('rf3', RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=30,n_estimators=60))
]

mlp_stack3 = StackingClassifier(estimators=est, final_estimator=RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50))
mlp_stack3.fit(np.array(x),np.array(y))

StackingClassifier(cv=None,
                   estimators=[('rf1',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=20,
                                                       min_samples_split=2,
                                   

In [0]:
## cross validation

train_df_cv = shuffle(train_df)
train_df_cv.reset_index(inplace=True, drop=True)
x_cv = train_df_cv.drop(feat1, axis=1)
y_cv = train_df_cv['target']


kf = KFold(n_splits=5)
i=1
tsum = 0
for train_index, test_index in kf.split(x):
    x_train, x_test = x_cv.loc[train_index[0]:train_index[-1]], x_cv.loc[test_index[0]:test_index[-1]]
    y_train, y_test = y_cv.loc[train_index[0]:train_index[-1]], y_cv.loc[test_index[0]:test_index[-1]]
    clf = StackingClassifier(estimators=est, final_estimator=RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50))
    clf.fit(np.array(x),np.array(y))
    y_pred = clf.predict(x_test)
    y_pred2 = clf.predict_proba(x_test)
    
    print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
    print(classification_report(y_test,y_pred))
    print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
    i=i+1
    tsum = tsum + log_loss(y_test,y_pred2)
    
print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")

log_loss for  1  :  0.6364936949776013 

              precision    recall  f1-score   support

           0       0.52      0.19      0.28       756
           1       0.77      0.93      0.84      6857
           2       0.48      0.28      0.35      2258

    accuracy                           0.72      9871
   macro avg       0.59      0.47      0.49      9871
weighted avg       0.68      0.72      0.69      9871

SVM Accuracy Score ->  72.36348900820585 for  1 

log_loss for  2  :  0.6376405123908137 

              precision    recall  f1-score   support

           0       0.53      0.17      0.26       788
           1       0.77      0.93      0.84      6915
           2       0.45      0.26      0.33      2168

    accuracy                           0.72      9871
   macro avg       0.58      0.46      0.48      9871
weighted avg       0.68      0.72      0.69      9871

SVM Accuracy Score ->  72.42427312329045 for  2 

log_loss for  3  :  0.6397480117700887 

              p

In [0]:
prediction = mlp_stack3.predict_proba(testing)
result = pd.DataFrame({'listing_id': test_df['listing_id'], 'high': prediction[:,0], 'medium': prediction[:,2], 'low': prediction[:,1] })
result.to_csv('m3_stack3.csv', index=False) 

In [0]:

# svc_class = SVC(kernel='linear', probability = True, cache_size=1000, )
# svc_class.fit(np.array(x), np.array(y))

# sclf = SVC(kernel='linear', probability = True, cache_size=1000)
# sclf.fit(x,y)
# rclf = RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50,min_samples_split=100)
# rclf.fit(x,y)
# mclf = MLPClassifier(alpha=1, max_iter=1000)
# mclf.fit(x,y)

# est = [
#        ('svm1', make_pipeline(MaxAbsScaler(),
#                              LinearSVC())),
#        ('svm2', make_pipeline(MaxAbsScaler(),
#                              linear_model.SGDClassifier()))  
# ]

# svc_class = StackingClassifier(estimators=est, final_estimator=SVC(kernel = 'linear', probability=True))
# svc_class.fit(np.array(x),np.array(y))

# svc_class = LinearSVC(random_state=0, tol=1e-5)
# svc_class.fit(x,y)
# svc_class = MLPClassifier(alpha = 0.1, hidden_layer_sizes = (50,50,50), max_iter=1000)
# svc_class.fit(np.array(x), np.array(y))
# svc_class = RandomForestClassifier(criterion="gini", random_state=0,min_samples_leaf=20,n_estimators=50,min_samples_split=100)
# svc_class.fit(np.array(x), np.array(y))

In [0]:
# ## Cross Validation

# train_df_cv = shuffle(train_df)
# train_df_cv.reset_index(inplace=True, drop=True)
# x_cv = train_df_cv.drop(feat1, axis=1)
# y_cv = train_df_cv['target']


# kf = KFold(n_splits=5)
# i=1
# tsum = 0
# for train_index, test_index in kf.split(x):
#     x_train, x_test = x_cv.loc[train_index[0]:train_index[-1]], x_cv.loc[test_index[0]:test_index[-1]]
#     y_train, y_test = y_cv.loc[train_index[0]:train_index[-1]], y_cv.loc[test_index[0]:test_index[-1]]
#     est = [
#        ('svm1', make_pipeline(MaxAbsScaler(),
#                              LinearSVC())),
#        ('svm2', make_pipeline(MaxAbsScaler(),
#                              linear_model.SGDClassifier()))  
#     ]
#     clf = StackingClassifier(estimators=est, final_estimator=SVC(kernel = 'linear', probability=True))
#     clf.fit(np.array(x_train),np.array(y_train))
#     y_pred = clf.predict(x_test)
#     y_pred2 = clf.predict_proba(x_test)
    
#     print('log_loss for ',i, ' : ', log_loss(y_test,y_pred2),'\n')
#     print(classification_report(y_test,y_pred))
#     print("SVM Accuracy Score -> ",accuracy_score(y_pred, y_test)*100, 'for ', i, '\n')
#     i=i+1
#     tsum = tsum + log_loss(y_test,y_pred2)
    
# print('log_loss average evaluation results of 5 classifiers: ', tsum/5 , "\n")